In [1]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [2]:
# Gather Data
boston_dataset = load_boston()
data = pd.DataFrame(data=boston_dataset.data, columns=boston_dataset.feature_names)
features = data.drop(['INDUS', 'AGE'], axis=1)

log_prices = np.log(boston_dataset.target)
target = pd.DataFrame(log_prices,columns=['PRICE'])

In [3]:
CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

property_stats = features.mean().values
property_stats = property_stats.reshape(1, 11)

In [4]:
type(features.mean())

pandas.core.series.Series

In [5]:
type(features.mean().values)

numpy.ndarray

In [6]:
regr = LinearRegression().fit(features, target)
fitted_vals = regr.predict(features)

# Challenge: calculate the MSE and RMSE using sklearn
MSE = mean_squared_error(target, fitted_vals)
RMSE = np.sqrt(MSE)

In [7]:
MSE

0.03516080084618688

In [8]:
def get_log_estimate(nr_rooms, students_per_classroom, next_to_river=False, high_confidence=True):
    
    # Configure property
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classroom
    
    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0
    
    # Make prediction
    log_estimate = regr.predict(property_stats)[0][0]
    
    # Calc Range 
    if high_confidence:
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68
    
    return log_estimate, upper_bound, lower_bound, interval

In [9]:
get_log_estimate(3, 20, next_to_river=True, high_confidence=False)

(2.776758191480399, 2.9642703266775294, 2.589246056283269, 68)

In [10]:
np.median(boston_dataset.target)

21.2

In [11]:
# Zillow is US based company for of real estate 

'''
Zillow Median Price is Obtained From It's Website
to define the scale factor of 
Modern Day Dollar Value And 1970's Dollar Values
'''

ZILLOW_MEDIAN_PRICE = 583.3
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE / np.median(boston_dataset.target)

log_est, upper, lower, conf = get_log_estimate(9, students_per_classroom=15, next_to_river=False, high_confidence=False)

# Convert Log Dollar price to today's dollars
dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
dollar_low = np.e**lower * 1000 * SCALE_FACTOR

# Round the dollar values to nearest thousand
rounded_est = np.around(dollar_est, -3)
rounded_hi = np.around(dollar_hi, -3)
rounded_low = np.around(dollar_low, -3)

print(f'The estimated property value is {rounded_est}.')
print(f'At {conf}% confidence the valuation range is')
print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end.')

The estimated property value is 827000.0.
At 68% confidence the valuation range is
USD 685000.0 at the lower end to USD 997000.0 at the high end.


# Let's create a function for Predict Boston House Price :

### Keyword's Arguments :

1. rm -> number of rooms in the property.
1. ptratio -> number of students per teacher in the classroom for the school in the area.
1. chas -> True if the property is next to the river, False otherwise.
1. large_range -> True for a 95% prediction interval, False for a 68% interval.


In [12]:
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):  
    
    if rm < 1 or ptratio < 1:
        print('That is unrealistic. Try again.')
        return

    log_est, upper, lower, conf = get_log_estimate(rm,students_per_classroom=ptratio,next_to_river=chas,high_confidence=large_range)

    # Convert to today's dollars
    dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
    dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
    dollar_low = np.e**lower * 1000 * SCALE_FACTOR

    # Round the dollar values to nearest thousand
    rounded_est = np.around(dollar_est, -3)
    rounded_hi = np.around(dollar_hi, -3)
    rounded_low = np.around(dollar_low, -3)

    print(f'The estimated property value is {rounded_est}.')
    print(f'At {conf}% confidence the valuation range is')
    print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end.')

In [13]:
get_dollar_estimate(rm=6, ptratio=60, chas=True)

The estimated property value is 130000.0.
At 95% confidence the valuation range is
USD 89000.0 at the lower end to USD 189000.0 at the high end.


# Miscellaneous

## Import function from boston_valuation.py file and call that for evaluation

In [15]:
import boston_valuation as val

In [16]:
val.get_dollar_estimate(6, 12, True)

The estimated property value is 783000.0.
At 95% confidence the valuation range is
USD 538000.0 at the lower end to USD 1139000.0 at the high end.


In [17]:
val.get_dollar_estimate(0, 12, True)

That is unrealistic. Try again.


In [20]:
val.get_dollar_estimate(9, 32, True)

The estimated property value is 486000.0.
At 95% confidence the valuation range is
USD 334000.0 at the lower end to USD 707000.0 at the high end.
